In [10]:
import cv2
import os
import numpy as np
import mediapipe as mp

# Path save
SAVE_PATH = r"C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences"
IMG_SIZE = 320
SEQUENCE_LENGTH = 30
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)


def normalize_landmarks(landmarks):
    x_coords = [lm[0] for lm in landmarks]
    y_coords = [lm[1] for lm in landmarks]
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)
    normalized_landmarks = [
        [(x - min_x) / (max_x - min_x), (y - min_y) / (max_y - min_y), z]
        for x, y, z in landmarks
    ]
    return normalized_landmarks


def preprocess_landmarks(landmarks, img_size=IMG_SIZE):
    normalized_landmarks = normalize_landmarks(landmarks)
    landmarks_image = np.zeros((img_size, img_size, 3), dtype=np.uint8)

    if normalized_landmarks:  # Check if landmarks are detected
        for connection in mp_hands.HAND_CONNECTIONS:
            start_idx = connection[0]
            end_idx = connection[1]
            x1, y1 = int(normalized_landmarks[start_idx][0] * (img_size - 1)), int(
                normalized_landmarks[start_idx][1] * (img_size - 1))
            x2, y2 = int(normalized_landmarks[end_idx][0] * (img_size - 1)), int(
                normalized_landmarks[end_idx][1] * (img_size - 1))
            cv2.line(landmarks_image, (x1, y1),
                     (x2, y2), (255, 255, 255), 2)

    for lm in normalized_landmarks:
        x, y = int(lm[0] * (img_size - 1)), int(lm[1] * (img_size - 1))
        cv2.circle(landmarks_image, (x, y), 5, (255, 0, 0), 3)

    return landmarks_image


gesture_name = input("Enter the gesture name to train: ").strip()
gesture_folder = os.path.join(SAVE_PATH, gesture_name)  # Folder for the gesture

cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print("Error: Camera not accessible.")
    exit()

sequence_count = 0
capturing = False
current_sequence = []

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame.")
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(landmark.x, landmark.y, landmark.z)
                         for landmark in hand_landmarks.landmark]
            preprocessed_image = preprocess_landmarks(landmarks)

            cv2.imshow('Preprocessed Hand', preprocessed_image)

            if capturing:
                current_sequence.append(preprocessed_image)

                if len(current_sequence) == SEQUENCE_LENGTH:
                    # Create sequence folder inside the gesture folder
                    sequence_folder = os.path.join(
                        gesture_folder,
                        f"{gesture_name}_{sequence_count}"
                    )
                    os.makedirs(sequence_folder, exist_ok=True)
                    
                    # Save frames inside the sequence folder
                    for i, img in enumerate(current_sequence):
                        frame_name = os.path.join(
                            sequence_folder,
                            f"frame_{i}.jpg"
                        )
                        cv2.imwrite(frame_name, img)
                    print(
                        f"Sequence {sequence_count} saved in folder: {sequence_folder}"
                    )

                    sequence_count += 1
                    current_sequence = []
                    capturing = False

    cv2.imshow('Hand Tracking', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):
        if not capturing:
            capturing = True
            print("Started capturing sequence...")
        else:
            capturing = False
            current_sequence = []
            print("Stopped capturing.")
    elif key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

Started capturing sequence...
Sequence 0 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_0
Started capturing sequence...
Sequence 1 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_1
Started capturing sequence...
Sequence 2 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_2
Started capturing sequence...
Sequence 3 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_3
Started capturing sequence...
Sequence 4 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_4
Started capturing sequence...
Sequence 5 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank you\Thank you_5
Started capturing sequence...
Sequence 6 saved in folder: C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences\Thank y